<a href="https://colab.research.google.com/github/mfmarlonferrari/tcc2020recommender/blob/main/KNN_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#https://pypi.org/project/mysql-connector/
!pip install mysql-connector flask-ngrok

In [174]:
import pandas as pd
import mysql.connector
from sklearn.neighbors import NearestNeighbors
import json

Conexão com a base de Dados transacional

In [205]:
def recomendacao(idUsuario):
  host="database-1.c6nlhkjxpqnn.sa-east-1.rds.amazonaws.com"
  port=3306
  dbname="DB_TCC"
  user="admin"
  password="admin123"

  conn = mysql.connector.connect(user=user, password=password,
                              host=host,
                              database=dbname)

  cur = conn.cursor()

  cur.execute('select * from questions')
  dados = cur.fetchall()
  dataset = pd.DataFrame(dados, columns=['ID', 'USUARIO', 'LIVRO', 'TITULO', 'RATING','D1','D2'])[['USUARIO','LIVRO','TITULO','RATING']]

  movies_features = dataset.pivot_table(index='USUARIO', columns='TITULO', values='RATING').fillna(0)

  modelo = NearestNeighbors(metric='cosine', n_neighbors=3)
  modelo.fit(movies_features)

  distancia, indices_vizinhos = modelo.kneighbors(movies_features.loc[id_usuario].values.reshape(1,-1), n_neighbors=4)

  parametros = {}

  for i in range(0, len(distancia.flatten())):
    if movies_features.index[indices_vizinhos.flatten()[i]] != id_usuario:
      parametros[movies_features.index[indices_vizinhos.flatten()[i]]] = distancia.flatten()[i]

  id_recomendado = movies_features.index[indices_vizinhos.flatten()][1]
  base = movies_features.loc[id_usuario].to_frame()
  recomendado = movies_features.loc[id_recomendado].to_frame()

  novos_titulos = pd.merge(base,recomendado,on='TITULO',how='inner')
  novos_titulos = novos_titulos[(novos_titulos[id_recomendado] > 0) & (novos_titulos[id_usuario] == 0)].reset_index()

  livros = json.loads(novos_titulos.to_json(force_ascii=False))
  parametros["livros"] = livros

  resultado_recomendacao = {"parametros":parametros}
  return str(resultado_recomendacao)

In [206]:
from flask_ngrok import run_with_ngrok
from flask import Flask
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/recomendar/<idUsuario>")
def recomendar(idUsuario):
    return recomendacao(idUsuario)
  
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0850ce90210a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Oct/2020 19:26:58] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2020 19:26:59] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2020 19:27:07] "GET /recomendar/4 HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2020 19:28:53] "GET /recomendar/4 HTTP/1.1" 200 -
